In [1]:
# Carga de librerías
import PIL
import PIL.Image
from PIL import Image
import pathlib
import scipy as sp
import seaborn as sns
from itertools import product
import pandas as pd
import time
import torch
import scipy.spatial
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os
import numpy as np  
import torch  
from torch.utils.data import Dataset, DataLoader  
import torchvision.transforms as transforms  
from torchvision.datasets import ImageFolder  
from PIL import Image  
import pathlib  
import cupy as cp 
from torchvision.datasets import ImageFolder
from torchvision import datasets

In [2]:
import pickle
import sys
from pathlib import Path

# Añadir el directorio raíz del proyecto a sys.path
module_path = Path('/Users/Cesar/Desktop/Proyecto-CSG/').resolve()
if module_path not in sys.path:
    sys.path.append(str(module_path))

# Añadir el directorio que contiene 'new_spectral_metric' a sys.path
new_spectral_metric_path = module_path / '/Users/Cesar/Desktop/Proyecto-CSG/pytorch_spectral_metric'
if new_spectral_metric_path not in sys.path:
    sys.path.append(str(new_spectral_metric_path))

from numpy.linalg import LinAlgError
from scipy.sparse.csgraph import laplacian
from pytorch_spectral_metric.estimator_pytorch import CumulativeGradientEstimator
from pytorch_spectral_metric.visualize_pytorch import make_graph
from pathlib import Path
from matplotlib.collections import LineCollection
from matplotlib.font_manager import FontProperties
from pathlib import Path

In [3]:
# Verificar la disponibilidad de la GPU  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print("GPU disponible:", torch.cuda.is_available())
print("Nombre de la GPU:", torch.cuda.get_device_name() if torch.cuda.is_available() else "N/A")


GPU disponible: True
Nombre de la GPU: NVIDIA GeForce RTX 4060


In [4]:
# Definir directorios
directorio_dataset = '../Datasets/CIFAR-10 dos clases'
imagenes_originales = '../Datasets/CIFAR-10 dos clases/cat'
imagenes_corruptas = '../Datasets/CIFAR-10 dos clases/cat_c_defocus_blur'

directorio_dataset = pathlib.Path(directorio_dataset)
imagenes_originales = pathlib.Path(imagenes_originales)
imagenes_corruptas = pathlib.Path(imagenes_corruptas)



# Dimensiones de imagen y tamaño de batch
img_height = 32
img_width = 32
#batch_size = len(list(data_dir.glob('*/*.png'))) # leer todas las imágenes al tiempo


# Transformaciones para preprocesamiento de datos
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),  # Ajustar tamaño a 32x32
    transforms.ToTensor(),  # Convertir a tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar
])

class CustomImageDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = pathlib.Path(directory)
        self.transform = transform
        self.images = list(self.directory.glob('*.png'))

    def __len__(self):  
        return len(self.images)  

    def __getitem__(self, idx):  
        img_path = self.images[idx]  
        image = Image.open(img_path).convert('RGB')  
        if self.transform:  
            image = self.transform(image)  
        return image


# Crear datasets personalizados
dataset_completo = datasets.ImageFolder(root=directorio_dataset, transform=transform)
dataset_originales = CustomImageDataset(imagenes_originales, transform=transform)
dataset_corruptas = CustomImageDataset(imagenes_corruptas, transform=transform)

# Calcular totales
total_dataset = len(dataset_originales) + len(dataset_corruptas)
total_originales = len(dataset_originales)
total_corruptas = len(dataset_corruptas)

batch_size = total_dataset



# Crear el DataLoader para iterar sobre los datos en lotes
# DataLoader para cargar los datasets
dataloader_completo = DataLoader(dataset_completo, batch_size=batch_size, shuffle=False, num_workers=0)
dataloader_originales = DataLoader(dataset_originales, batch_size=batch_size, shuffle=False)
dataloader_corruptas = DataLoader(dataset_corruptas, batch_size=batch_size, shuffle=False)

# # DataLoader para cargar los datasets  
# dataloader_completo = DataLoader(dataset_completo, batch_size=batch_size, shuffle=False, num_workers=0)  

print(f"Total imágenes dataset: {total_dataset}")  
print(f"Total imágenes originales: {total_originales}")  
print(f"Total imágenes corruptas: {total_corruptas}")  


Total imágenes dataset: 10000
Total imágenes originales: 5000
Total imágenes corruptas: 5000


In [5]:
# Acceder a las clases

class_names = dataset_completo.classes
num_classes = len(class_names)

print(f"Número de clases: {num_classes}")
print(f"Los nombres de las clases son: {class_names}")

Número de clases: 2
Los nombres de las clases son: ['cat', 'cat_c_defocus_blur']


In [6]:
# Iterar sobre los datos para obtener el primer lote  
for images, labels in dataloader_completo:  
    # Mover datos a la GPU si es posible  
    images, labels = images.to(device), labels.to(device)  
    
    # Si deseas mantener las imágenes en su forma original (no aplanadas), simplemente omite la aplanación  
    if device.type == 'cuda':  
        numpy_all_images = cp.array(images.cpu().numpy())  # Mantiene la forma original  
        numpy_all_labels = cp.array(labels.cpu().numpy())  
    else:  
        numpy_all_images = cp.array(images.numpy())  # Mantiene la forma original  
        numpy_all_labels = cp.array(labels.numpy())  
    
    print(numpy_all_images.shape)  # Esto debería imprimir (10000, 3, 32, 32)  
    break  # Solo tomamos el primer lote para imprimir las dimensiones  

(10000, 3, 32, 32)


In [7]:
# Inicializar el estimador  
estimator = CumulativeGradientEstimator(M_sample=348, k_nearest=10)  

# Convertir a un ndarray de NumPy si numpy_all_images o numpy_all_labels son de tipo cp.ndarray  
data_np = numpy_all_images.get() if isinstance(numpy_all_images, cp.ndarray) else numpy_all_images  
target_np = numpy_all_labels.get() if isinstance(numpy_all_labels, cp.ndarray) else numpy_all_labels  

# Ajustar el modelo con los datos y etiquetas convertidos  
estimator.fit(data=data_np, target=target_np)  
csg = estimator.csg  # The actual complexity values.  
estimator.evals, estimator.evecs  # The eigenvalues and vectors.  

# Asegúrate de que tienes un entorno gráfico activo al usar esta función  
make_graph(estimator.difference, title=f"Comparative between two classes, CSG: {csg}", classes=class_names)

Type of k: <class 'cupy.ndarray'>, Value of k: 0
Type of k: <class 'cupy.ndarray'>, Value of k: 1


ValueError: Found array with dim 4. check_pairwise_arrays expected <= 2.